In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import os
from pandas.io.formats.excel import ExcelFormatter
from pandas.io import gbq
from openpyxl.workbook import Workbook
from shutil import copyfile
from pathlib import Path

In [2]:
old_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_0"
old_parquet_files = glob.glob(os.path.join(old_path, "*.parquet"))
new_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_1"
new_parquet_files = glob.glob(os.path.join(new_path, "*.parquet"))

In [3]:
for file in old_parquet_files:
    df = (pd.read_parquet(file))
    UniqueNames = df.Identifier.unique()
    DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}
    for key in DataFrameDict.keys():
        DataFrameDict[key] = df[:][df.Identifier == key]
        df_1= DataFrameDict[key]
        start_col_index = 2  # Assuming the conversion should start from the second column
        end_col_index = 12   # Assuming 8 consecutive columns need to be converted
        filepath = new_path + '/' + key + '.parquet'
        # Convert True and False to 1 and 0 in the specified column range
        df.iloc[:, start_col_index:end_col_index] = df.iloc[:, start_col_index:end_col_index].astype(float)
        df.iloc[:, start_col_index:end_col_index] = df.iloc[:, start_col_index:end_col_index].replace({True: 1, False: 0})

        df['date_time'] = df['date_time'].dt.tz_localize('UTC')
        df['time_cvt'] = df['date_time'].dt.tz_convert('US/Eastern')
        df['time'] = df['time_cvt'].dt.strftime('%Y-%m-%d %H:%M:%S')
        df =df.drop('time_cvt', axis=1)
        df['year'] = df['date_time'].dt.year.astype('int16')  # Downcast to smaller integer type
        df['month'] = df['date_time'].dt.month.astype('int8')  # Downcast to smaller integer type
        df['day'] = df['date_time'].dt.day.astype('int8')  # Downcast to smaller integer type
        df['time'] = df['date_time'].dt.time  # No change needed
        df.to_parquet(filepath)